# EarthRanger IO

## Setup

### Ecoscope

In [ ]:
%pip install \
    'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v2.10.7' &> /dev/null

In [ ]:
import getpass
import os
import sys

import pandas as pd

import ecoscope

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

### Connect to EarthRanger

In [ ]:
ER_SERVER = os.getenv("ER_SERVER", "https://mep-dev.pamdas.org")
ER_USERNAME = os.getenv("ER_USERNAME", "username")  # replace with your username

ER_PASSWORD = os.getenv("ER_PASSWORD", "")

if not ER_PASSWORD:
    ER_PASSWORD = getpass.getpass("Please enter your ER password: ")

er_io = ecoscope.io.EarthRangerIO(
    server=ER_SERVER,
    username=ER_USERNAME,
    password=ER_PASSWORD,
    tcp_limit=5,
    sub_page_size=4000,
)

## GET `Event`

### By `event_ids`
Use this approach to download an individual `Event` if you know its id. Downloading from multiple ids at once is not currently supported in ER.

In [ ]:
er_io.get_events(
    event_ids="76e76c8c-0936-4ef4-9298-0cbd4f516a9c",  # Distance count event
)

### By `event_type`
Use this approach if you want to download events of a given event type (using the EventType ID from ER)

In [ ]:
df_event_types = pd.DataFrame(er_io.get_event_types())
mepdev_distance_count_id = df_event_types[df_event_types["value"] == "mepdev_distance_count"]["id"].values[0]
mepdev_distance_count_id

In [ ]:
er_io.get_events(
    event_type=[mepdev_distance_count_id],  # MEP Distance Count
    since=pd.Timestamp("2023-01-01").isoformat(),
    until=pd.Timestamp("2024-01-01").isoformat(),
).reset_index()

### Unpack `event_details` column of `JSON` data into individual columns

In [ ]:
df_event_types = pd.DataFrame(er_io.get_event_types())
mepdev_distance_survey_id = df_event_types[df_event_types["value"] == "mepdev_distance_count"]["id"].values[0]
mepdev_distance_survey_id

In [ ]:
events_df = er_io.get_events(event_type=[mepdev_distance_survey_id])  # MEP Distance Survey Count
events_df.event_details

In [ ]:
ecoscope.io.earthranger_utils.normalize_column(events_df, "event_details")
events_df

## GET `Subjects`

### by `id`

In [ ]:
df_subjects = er_io.get_subjects()
habiba_id = df_subjects[df_subjects["name"] == "Habiba"]["id"].values[0]
habiba_id

In [ ]:
er_io.get_subjects(id=habiba_id)  # Habiba

### By `SubjectGroup` `subject_group_name`

In [ ]:
er_io.get_subjects(subject_group_name="Elephants")

## GET `Observations`

The output is an Ecoscope `Relocations` dataframe that has a core set of columns: `groupby_col`, `fixtime`, `junk_status`.

All other columns are prepended by `extra__`. The `groupby_col` will be the `subject_id` and the index of the dataframe is the observation indices.

Classic ER `Observation` format can be returned by passing in parameter `relocations=False`.

Filter observation by setting `filter` to one of: None, 0, 1, 2 or 3
- None returns everything
- 0 filters out everything but observations with exclusion flag 0 (Pass back clean data)
- 1 filters out everything but observations with exclusion flag 1 (Pass back manually filtered data)
- 2 filters out everything but observations with exclusion flag 2 (Ppass back automatically filtered data)
- 3 filters out everything but observations with exclusion flag 2 or 1 (Pass back both manual and automatic filtered data)

### By `SubjectGroup` `subject_group_name`:

In [ ]:
since = pd.Timestamp("2008-01-01").isoformat()
until = pd.Timestamp("2020-01-01").isoformat()

In [ ]:
relocs = er_io.get_subjectgroup_observations(
    subject_group_name="Elephants",
    filter=0,
    since=since,
    until=until,
    include_details=True,
    include_inactive=True,
    include_source_details=True,
    include_subject_details=True,
    include_subjectsource_details=True,
)
relocs.gdf

### By `subject_id`

In [ ]:
df_subjects = er_io.get_subjects()
habiba_id = df_subjects[df_subjects["name"] == "Habiba"]["id"].values[0]
salif_keita_id = df_subjects[df_subjects["name"] == "Salif Keita"]["id"].values[0]
habiba_id, salif_keita_id

In [ ]:
relocs = er_io.get_subject_observations(
    subject_ids=[
        habiba_id,  # Habiba
        salif_keita_id,  # Salif Keita
    ],
    since=since,
    until=until,
    filter=0,
    include_details=True,
    include_subject_details=True,
    include_source_details=True,
    include_subjectsource_details=True,
)
relocs.gdf

### By `source_id`

In [ ]:
df_sources = er_io.get_sources()
habiba_source_id = df_sources[df_sources["manufacturer_id"] == "ST-Habiba"]["id"].values[0]
salif_keita_source_id = df_sources[df_sources["manufacturer_id"] == "ST-SalifKeita"]["id"].values[0]
habiba_source_id, salif_keita_source_id

In [ ]:
df_sources = er_io.get_sources()

relocs = er_io.get_source_observations(
    source_ids=[
        habiba_source_id,  # Habiba's Source
        salif_keita_source_id,  # Salif Keita's Source
    ],
    since=since,
    until=until,
    filter=0,
    include_details=True,
    include_source_details=True,
)
relocs.gdf

## GET `Patrols`

In [ ]:
df_patrol_types = er_io.get_patrol_types().reset_index()
mep_distance_id = df_patrol_types[df_patrol_types["value"] == "MEP_Distance_Survey_Patrol"]["id"].values[0]
mep_distance_id

In [ ]:
er_io.get_patrols(patrol_type=[mep_distance_id])  # MEP Distance Count Survey

## GET `Observations` for a `Patrol`

In [ ]:
patrol_df = er_io.get_patrols(
    since=pd.Timestamp("2017-01-01").isoformat(),
    until=pd.Timestamp("2017-04-01").isoformat(),
)


relocs = er_io.get_patrol_observations(
    patrol_df,
    include_patrol_details=True,
    include_source_details=True,
    include_subject_details=True,
    include_subjectsource_details=True,
)
relocs.gdf

## GET `SPATIAL FEATURES`

### GET `SPATIAL FEATURE` from `spatial_feature_id`

In [ ]:
spatial_feature = er_io.get_spatial_feature(spatial_feature_id="8868718f-0154-45bf-a74d-a66706ef958f")
spatial_feature

### GET `SPATIAL FEATURES` from `spatial_features_group_id`

In [ ]:
spatial_features = er_io.get_spatial_features_group(spatial_features_group_id="15698426-7e0f-41df-9bc3-495d87e2e097")
spatial_features

## GET `Users`

In [ ]:
df = pd.DataFrame(er_io.get_users())
df

## Export to File

### Create test data

Skip this cell if you would like to provide your own data

In [ ]:
relocs = er_io.get_subjectgroup_observations(
    subject_group_name="Elephants",
    filter=0,
    since=since,
    until=until,
    include_details=True,
    include_inactive=True,
    include_source_details=True,
    include_subject_details=True,
    include_subjectsource_details=True,
)
relocs.gdf

### GeoPackage (.gpkg)
(GeoPackage does not support columns with type `list` so we `drop` them.)

In [ ]:
relocs_gdf = relocs.gdf.select_dtypes(exclude=[list])

relocs_gdf.to_file(os.path.join(output_dir, "observations.gpkg"), layer="observations")

### CSV

In [ ]:
relocs_gdf.to_csv(os.path.join(output_dir, "observations.csv"))

### Python Pickle (.pkl)

In [ ]:
relocs_gdf.to_pickle(os.path.join(output_dir, "observations.pkl"))

### Apache Feather (.feather)

In [ ]:
relocs_gdf.to_feather(os.path.join(output_dir, "observations.feather"))